In [28]:
import numpy as np
import math
from tracklet import Tracklet
from tracklet import TrackletCollection
from tracklet import TrackletGT
from tracklet import parse_xml
import mayavi.mlab

# Generate tracklet

In [20]:
pred_box = np.load('./one_frame_data/gt_boxes3d.npy')[0]
print pred_box
print pred_box.shape

[[ 11.33253098   4.20234966  -1.74235904]
 [ 12.27725887   2.64294267  -1.74235904]
 [ 16.06950378   4.9403801   -1.74235904]
 [ 15.12477589   6.49978685  -1.74235904]
 [ 11.33253098   4.20234966   0.25764096]
 [ 12.27725887   2.64294267   0.25764096]
 [ 16.06950378   4.9403801    0.25764096]
 [ 15.12477589   6.49978685   0.25764096]]
(8, 3)


In [21]:
def box_to_tracklet(box, frame_number):
    lv2d = box[1][:2] - box[0][:2] # x,y component of l vector
    l = np.linalg.norm(lv2d)
    w = np.linalg.norm(box[3][:2] - box[0][:2])
    h = box[4][2] - box[0][2]
    center = (box[0] + box[6]) * 0.5
    lv2dn = lv2d / l # normalize
    yaw = 0
    if lv2dn[0] < 0.0001:
        yaw = math.pi if lv2dn[1] > 0 else -math.pi
    else:
        yaw = math.atan2(lv2dn[1], lv2dn[0])    
    t = Tracklet('Car', l, w, h)
    t.first_frame = frame_number
    p = {'tx': center[0], 'ty': center[1], 'tz': center[2], 'rx': 0, 'ry': 0, 'rz': yaw}
    t.poses.append(p)
    return t

In [22]:
def write_tracklet(filename, box_list):
    tracklet_col = TrackletCollection()
    for i in range(len(box_list)):
        t = box_to_tracklet(box_list[i], i)
        tracklet_col.tracklets.append(t)
        tracklet_col.write_xml(filename)

In [22]:
write_tracklet('./one_frame_data/tracklet.xml', pred_box)

# Visualize lidar points, bounding box, and ground truth tracklet

In [23]:
lidar = np.load('./one_frame_data/lidar.npy')
print lidar.shape

(123397, 4)


In [25]:
def tracklet_gt_to_box(filename, tracklet_idx, frame_number):
    tracklet_gt = parse_xml(filename)[tracklet_idx]
    h = tracklet_gt.size[2]
    w = tracklet_gt.size[1]
    l = tracklet_gt.size[0]
    bbox = np.array([
        [-l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2],
        [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2],
        [-h / 2, -h / 2, -h / 2, -h / 2, h / 2, h / 2, h / 2, h / 2],
    ])
    yaw = tracklet_gt.rots[frame_number][2]
    rot_mat = np.array([
        [np.cos(yaw), -np.sin(yaw), 0.0],
        [np.sin(yaw), np.cos(yaw), 0.0],
        [0.0, 0.0, 1.0]
    ])
    position = tracklet_gt.trans[frame_number]
    oriented_bbox = np.dot(rot_mat, bbox) + np.tile(position, (8, 1)).T
    gt_box = oriented_bbox.T
    return gt_box

In [26]:
gt_box = tracklet_gt_to_box('./one_frame_data/tracklet_gt.xml', 0, 1000)
print gt_box

Parsing Tracklet file ./one_frame_data/tracklet_gt.xml
File contains 1 Tracklets
Loaded 1 Tracklets
[[ 8.203899  0.761408 -2.590839]
 [ 8.203899 -0.686392 -2.590839]
 [ 9.778699 -0.686392 -2.590839]
 [ 9.778699  0.761408 -2.590839]
 [ 8.203899  0.761408  1.650961]
 [ 8.203899 -0.686392  1.650961]
 [ 9.778699 -0.686392  1.650961]
 [ 9.778699  0.761408  1.650961]]


In [31]:
def viz_point_box_groundtruth(points, pred_boxes, gt_boxes, vals="distance"):
    x = points[:, 0]  # x position of point
    y = points[:, 1]  # y position of point
    z = points[:, 2]  # z position of pointfrom mpl_toolkits.mplot3d import Axes3D
    # r = lidar[:, 3]  # reflectance value of point
    d = np.sqrt(x ** 2 + y ** 2)  # Map Distance from sensor

    # Plot using mayavi -Much faster and smoother than matplotlib
    #import mayavi.mlab
    if vals == "height":
        col = z
    else:
        col = d

    fig = mayavi.mlab.figure(bgcolor=(0, 0, 0), size=(640, 360))
    mayavi.mlab.points3d(x, y, z,
                         col,          # Values used for Color
                         mode="point",
                         colormap='spectral', # 'bone', 'copper', 'gnuplot'
                         # color=(0, 1, 0),   # Used a fixed (r,g,b) instead
                         figure=fig,
                         )
    
    # predicted boxes
    pred_color = (1,1,1)
    for i in range(len(pred_boxes)):
        car = pred_boxes[i]
        x = car[:,0]
        y = car[:,1]
        z = car[:,2]

        mayavi.mlab.plot3d(x[:4], y[:4], z[:4], tube_radius=0.025, color=pred_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[0,3]], y[[0,3]], z[[0,3]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[0,4]], y[[0,4]], z[[0,4]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[1,5]], y[[1,5]], z[[1,5]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[2,6]], y[[2,6]], z[[2,6]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[3,7]], y[[3,7]], z[[3,7]], tube_radius=0.025, color=pred_color)
        
        mayavi.mlab.plot3d(x[-4:], y[-4:], z[-4:], tube_radius=0.025, color=pred_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[4,7]], y[[4,7]], z[[4,7]], tube_radius=0.025, color=pred_color)
    
    #ground truth boxes
    gt_color = (1,1,0)
    for i in range(len(gt_boxes)):
        car = gt_boxes[i]
        x = car[:,0]
        y = car[:,1]
        z = car[:,2]

        mayavi.mlab.plot3d(x[:4], y[:4], z[:4], tube_radius=0.025, color=gt_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[0,3]], y[[0,3]], z[[0,3]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[0,4]], y[[0,4]], z[[0,4]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[1,5]], y[[1,5]], z[[1,5]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[2,6]], y[[2,6]], z[[2,6]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[3,7]], y[[3,7]], z[[3,7]], tube_radius=0.025, color=gt_color)
        
        mayavi.mlab.plot3d(x[-4:], y[-4:], z[-4:], tube_radius=0.025, color=gt_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[4,7]], y[[4,7]], z[[4,7]], tube_radius=0.025, color=gt_color)
        
    mayavi.mlab.show()

In [32]:
pred_boxes = []
gt_boxes = []

pred_boxes.append(pred_box)
gt_boxes.append(gt_box)
viz_point_box_groundtruth(lidar, pred_boxes, gt_boxes)